### 4.1 Saliency Mapping - Feature Occlusion

Feature occlusion studies were performed to show the influence of occluding regions of input image to the confidence score predicted by the CNN model. 

The occlusion map was computed by replacing a region of the image with a pure white patch and generating a prediction on the occluded image. A sliding window approach was applied on the 256x256 pixels input images. A white patch was systematically slided across the image to replace a specific region of the input image, the CNN model forward propagated and generated the prediction confidence score on the occluded image at each time. As systematically sliding the white patch across the image (stride = 1 pixel), the prediction confidence score on the occluded image was recorded as each pixel of the occlusion map. The results were ploted as heatmaps, with red being the most confidence, and blue the least.

In [1]:
import os, time

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import PIL.Image as Image

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

In [2]:
IMG_DIR = 'data/tiles/hold-out/'
MODEL_DIR = 'models/CNN_model_parameters.pkl'
SAVE_DIR = 'data/outputs/selected_test_blobs/'

In [3]:
if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

In [4]:
image_classes = ['cored', 'diffuse', 'CAA']
use_gpu = torch.cuda.is_available()

norm = np.load('utils/normalization.npy').item()

In [7]:
file = pd.read_csv('data/CSVs/selected_test_blobs.csv')
image_list = list(file['tilename'])

In [4]:
def imshow(inp, ax=plt, title=None, pause=True):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array(norm['mean'])
    std = np.array(norm['std'])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    
    ax.imshow(inp)
    if title is not None:
        try:
            ax.title(title)
        except:
            ax.set(title=title)
    if pause:
        plt.pause(0.001)

In [ ]:
class Net(nn.Module):

    def __init__(self, fc_nodes=1024, num_classes=3, dropout=0.5):
        super(Net, self).__init__()

    def forward(self, x):
 
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

In [ ]:
# instatiate the model 
model = torch.load(MODEL_DIR, map_location=lambda storage, loc: storage)

if use_gpu:
    model = model.cuda()

In [6]:
def single_prediction(img_name, verbose=False):
    """
        Perform forward propagation on a single image 
    """
    im = Image.open(IMG_DIR+img_name)
    imtensor = transforms.ToTensor()(im)
    imtensor = transforms.Normalize(norm['mean'], norm['std'])(imtensor)
    imtensor = imtensor.view(1,imtensor.shape[0],imtensor.shape[1],imtensor.shape[2])
    output = F.sigmoid(model.module(Variable(imtensor.cuda())))
    if verbose:
        print(output)

In [7]:
def occluded_predictions(img_name, block_size=32, verbose=False):
    """
        Sliding window approach to perform feature occlusion study.
        A white patch was systematically slided across the image to replace a region of the image,
        performing forward propagation on the occluded image at each time.
    """
    
    im = Image.open(IMG_DIR+img_name)
    imtensor = transforms.ToTensor()(im)
    
    stride = 1
    final_output = torch.zeros(3, (imtensor.shape[1]-block_size)//stride+1, (imtensor.shape[2]-block_size)//stride+1)

    start = time.time()
    model.train(False)
    for row in range(0, imtensor.shape[1]-block_size+1, stride):
        for col in range(0, imtensor.shape[2]-block_size+1, stride):
            imtensor = transforms.ToTensor()(im)
            imtensor[:,row:row+block_size,col:col+block_size] = torch.ones(3,block_size,block_size)
            imtensor = transforms.Normalize(norm['mean'], norm['std'])(imtensor)
            imtensor = imtensor.view(1,imtensor.shape[0],imtensor.shape[1],imtensor.shape[2])
            output = F.sigmoid(model.module(Variable(imtensor.cuda())))
            final_output[:,row//stride,col//stride] = output.data.cpu()[0]
    end = time.time()
    
    if verbose:
        print('time: {}m {}s'.format((end-start)//60, (end-start)%60))
    
    return final_output

In [8]:
def show_heatmaps(img_name, block_size, final_output, path=None, show=False, verbose=False):
    """
        plot and save the heatmaps
    """
    if verbose:
        print('size:', block_size)
    
    im = Image.open(IMG_DIR+img_name)
    imtensor = transforms.ToTensor()(im)
    imtensor[:,-block_size:,-block_size:] = torch.ones(3,block_size,block_size)
    imtensor = transforms.Normalize(norm['mean'], norm['std'])(imtensor)
    
    colormap = 'RdYlBu_r'

    if show:
        fig = plt.figure(figsize=(21,5))

        ax = fig.add_subplot(141)
        imshow(imtensor, title=img_name, pause=False)

        ax = fig.add_subplot(142)
        ax.set(title='cored')
        im1 = ax.imshow(final_output[0], cmap=colormap, vmin=0., vmax=1.)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        plt.colorbar(im1, cax=cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

        ax = fig.add_subplot(143)
        ax.set(title='Diffuse')
        im2 = ax.imshow(final_output[1], cmap=colormap, vmin=0., vmax=1.)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        plt.colorbar(im2, cax=cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

        ax = fig.add_subplot(144)
        ax.set(title='CAA')
        im3 = ax.imshow(final_output[2], cmap=colormap, vmin=0., vmax=1.)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        plt.colorbar(im3, cax=cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
        
        plt.pause(0.01)
    
        if path:
            fig.savefig(path+'ablations.jpg')
    
    if path:
        plt.imsave(path+'ablation_size-{}_cored.png'.format(block_size), 
                   final_output[0], cmap=colormap, vmin=0., vmax=1.)
        plt.imsave(path+'ablation_size-{}_diffuse.png'.format(block_size), 
                   final_output[1], cmap=colormap, vmin=0., vmax=1.)
        plt.imsave(path+'ablation_size-{}_caa.png'.format(block_size), 
                   final_output[2], cmap=colormap, vmin=0., vmax=1.)

In [10]:
for img in image_list:
    if img is np.nan:
        continue
        
    wsi_name = img.split('/')[0]
    source_name = ''.join(img.split('/')[-1].split('.jpg'))
    img_name = wsi_name+'/'+source_name+'.jpg'
    save_path = SAVE+DIR+'{}/'.format(source_name)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    single_prediction(img_name)

    block_sizes = [2, 4, 8, 16, 32, 64]

    for block_size in block_sizes:
        final_output = occluded_predictions(img_name, block_size)
        np.save(save_path+'ablation_size-{}'.format(block_size), final_output)
        
        show_heatmaps(img_name, block_size, final_output, path=save_path)
        
print('done')

done
